In [1]:
import pybamm as pb;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False

import openpyxl

# Please read:  2022-08-03
This back-up script is to reproduce the errors occur for Coupled_RioP2 branch 
when I try to run full GEM model with OKANE2022 para and break-in Total_Cycles
which is:
Experiment_Breakin_0 = pb.Experiment( [
    ( f"Discharge at 0.1C until {V_min} V",  
    "Rest for 6 hours",  
    f"Charge at 0.1C until {V_max} V" )  ] *2 )

Errors occur when I rest for 6 hours in the 2nd cycle

In [2]:
# Test this branch - Coupled_RioP2 newly created from Simon's develop branch
# Run without dry-out model
V_max = 4.2;        V_min = 2.5;   Temper_RPT = 25; 
Total_Cycles = 10; save_at_cycles = 1;  
for i in range(0,1):  # Key scan parameters:
    Diff_SEI_i = 1.7e-20; 
    R_SEI_i = 2E5;   Bulk_Sol_Con_i = 4541.0;
    D_Li_inSEI_i = 1e-12;    # default: 1e-20 
    c_Li_inte_ref_i = 15;    # default: 15
    Diff_EC_i   = 2e-20;       k_SEI_i   = 1e-16;
    LAMcr_prop_i=2.7778e-07;   Crack_rate_i=3.9e-20; 
    Couple_SEI_LiP_i = 1e-6; # default: 1e-6
    k_LiP_i = 1E-10;         # default: 1e-10
    Temper = 25;
    mesh_par = 120;
     
for i in range(0,1):  
    ChemistryChen=pb.parameter_sets.OKane2022   
    #ChemistryChen["electrolyte"] = "lipf6_Valoen2005"
    Para_0=pb.ParameterValues(chemistry=ChemistryChen)
    # DFN parameter
    Para_0.update({"Upper voltage cut-off [V]": 4.21})
    Para_0.update({"Lower voltage cut-off [V]": 2.49})
    Para_0.update({"Negative electrode diffusivity [m2.s-1]": 2e-13})
    Para_0.update({"Positive electrode diffusivity [m2.s-1]": 1e-13})
    # general SEI
    Para_0.update({'SEI resistivity [Ohm.m]':R_SEI_i}) ;
    # solvent-diffusion limited
    Para_0.update({'Outer SEI solvent diffusivity [m2.s-1]':Diff_SEI_i});
    Para_0.update({'Bulk solvent concentration [mol.m-3]':Bulk_Sol_Con_i});
    # interstitial-diffusion limited
    Para_0.update({'Inner SEI lithium interstitial diffusivity [m2.s-1]':D_Li_inSEI_i})       
    Para_0.update({'Lithium interstitial reference concentration [mol.m-3]':c_Li_inte_ref_i}) 
    # ec-reaction limited
    Para_0.update({'EC diffusivity [m2.s-1]':Diff_EC_i}) 
    Para_0.update({'SEI kinetic rate constant [m.s-1]':k_SEI_i}) 
    Para_0.update({'EC initial concentration in electrolyte [mol.m-3]':Bulk_Sol_Con_i}) 
    # LiP and coupling with SEI:
    Para_0.update({"Dead lithium decay constant [s-1]": Couple_SEI_LiP_i})   
    Para_0.update({'Lithium plating kinetic rate constant [m.s-1]':k_LiP_i})  
    # Crack model
    Para_0.update({"Negative electrode LAM constant proportional term [s-1]": LAMcr_prop_i})
    Para_0.update({"Positive electrode LAM constant proportional term [s-1]": LAMcr_prop_i})  
    # make it simple for now, but may want to have T dependency in the future
    Para_0.update({"Negative electrode cracking rate": Crack_rate_i})
    Para_0.update({"Positive electrode cracking rate": Crack_rate_i})
    Para_0.update({"Negative electrode volume change": 0.0})
    Para_0.update({"Positive electrode volume change": 0.0}) 
    Model_0 = pb.lithium_ion.DFN(     
        options={
            "particle": "Fickian diffusion",          
            "SEI":"solvent-diffusion limited",   
            "SEI on cracks":"true",  
            "SEI film resistance":"distributed",          
            "SEI porosity change":"true",      
            "particle mechanics":"swelling and cracking",  
            "loss of active material":"stress-driven", 
            "lithium plating":"partially reversible"      } ) #
    var = pb.standard_spatial_vars  
    var_pts = {
        var.x_n: 20,  
        var.x_s: 10,  
        var.x_p: 20,  
        var.r_n: mesh_par,  
        var.r_p: mesh_par,  }

In [3]:
# Run break-in cycle step by step:
Experiment_Breakin_0 = pb.Experiment( [
    ( f"Discharge at 0.1C until {V_min} V",  
    "Rest for 6 hours",  
    f"Charge at 0.1C until {V_max} V" )  ] *1 )
Experiment_Breakin_1 = pb.Experiment( [
    ( f"Discharge at 0.1C until {V_min} V",  )  ] *1 )
Experiment_Breakin_2 = pb.Experiment( [
    ( "Rest for 1 hour",  )  ] *1 )

# Run 1st cycle
Sim_Breakin_0    = pb.Simulation(
    Model_0,        experiment = Experiment_Breakin_0,
    parameter_values = Para_0,
    solver = pb.CasadiSolver(),
    var_pts=var_pts,
    ) 
Sol_Breakin_0    = Sim_Breakin_0.solve(calc_esoh=False)
# Run 2nd cycle - diacharge
Model_Breakin_1 = Model_0.set_initial_conditions_from(Sol_Breakin_0, inplace=False)

Sim_Breakin_1    = pb.Simulation(
    Model_Breakin_1,        experiment = Experiment_Breakin_1,
    parameter_values = Para_0,
    solver = pb.CasadiSolver(),
    var_pts=var_pts,
    ) 
Sol_Breakin_1    = Sim_Breakin_1.solve(calc_esoh=False)


psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
At t = 5.28792e-006 

In [4]:
# Run 2nd cycle - rest 1h
Model_Breakin_2 = Model_Breakin_1.set_initial_conditions_from(Sol_Breakin_1, inplace=False)

Sim_Breakin_2    = pb.Simulation(
    Model_Breakin_2,        experiment = Experiment_Breakin_2,
    parameter_values = Para_0,
    solver = pb.CasadiSolver(),
    var_pts=var_pts,
    ) 
Sol_Breakin_2    = Sim_Breakin_2.solve(calc_esoh=False)


In [5]:
Experiment_Breakin_3 = pb.Experiment( [
    ( "Rest for 2 min",  )  ] *1 )
Model_Breakin_3 = Model_Breakin_2.set_initial_conditions_from(Sol_Breakin_2, inplace=False)

Sim_Breakin_3    = pb.Simulation(
    Model_Breakin_3,        experiment = Experiment_Breakin_3,
    parameter_values = Para_0,
    solver = pb.CasadiSolver(),
    var_pts=var_pts,
    ) 
Sol_Breakin_3    = Sim_Breakin_3.solve(calc_esoh=False)


psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed


In [6]:
Experiment_Breakin_4 = pb.Experiment( [
    ( "Rest for 0.01 s",  )  ] *1 )
Model_Breakin_4 = Model_Breakin_3.set_initial_conditions_from(Sol_Breakin_3, inplace=False)

Sim_Breakin_4    = pb.Simulation(
    Model_Breakin_4,        experiment = Experiment_Breakin_4,
    parameter_values = Para_0,
    solver = pb.CasadiSolver(),
    var_pts=var_pts,
    ) 
Sol_Breakin_4    = Sim_Breakin_4.solve(calc_esoh=False)


psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
The residual routine or the linear setup or solve routine had a recoverable error, but IDACalcIC was unable to recover.
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian faile

SolverError: Maximum number of decreased steps occurred at t=0.0. Try solving the model up to this time only or reducing dt_max (currently, dt_max=3.5458164923888056).

In [7]:
print(Sol_Breakin_3["Terminal voltage [V]"].entries)
print(Sol_Breakin_2["Terminal voltage [V]"].entries)

[2.54307523 2.54303804 2.54300086]
[2.53508658 2.5444107  2.54502107 2.54513074 2.54513457 2.54511244
 2.54508259 2.54504956 2.54501487 2.5449792  2.5449429  2.54490619
 2.54486921 2.54483204 2.54479475 2.54475738 2.54471995 2.54468248
 2.54464499 2.54460748 2.54456996 2.54453245 2.54449493 2.54445741
 2.5444199  2.5443824  2.5443449  2.54430741 2.54426993 2.54423245
 2.54419499 2.54415753 2.54412008 2.54408264 2.54404522 2.54400779
 2.54397038 2.54393298 2.54389559 2.5438582  2.54382083 2.54378346
 2.5437461  2.54370876 2.54367142 2.54363409 2.54359677 2.54355946
 2.54352215 2.54348486 2.54344757 2.5434103  2.54337303 2.54333578
 2.54329853 2.54326129 2.54322406 2.54318684 2.54314962 2.54311242
 2.54307523]


In [8]:
label = ["2","3"] 
output_variables3 = [
                "Electrolyte potential [V]",
                "Positive electrode potential [V]",
                "Negative electrode potential [V]",
                "Positive current collector potential [V]", 
                "Terminal voltage [V]", 
            ]
quick_plot = pb.QuickPlot([Sol_Breakin_2,Sol_Breakin_3], output_variables3,label,variable_limits='tight') #     
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

In [9]:
label = ["2","3"] 
output_variables3 = [
                "Minimum positive particle concentration",
                "Minimum negative particle concentration",
                "Maximum positive particle concentration",
                "Maximum negative particle concentration",
                "Maximum positive particle surface concentration",
                "Maximum negative particle surface concentration",
            ]
quick_plot = pb.QuickPlot([Sol_Breakin_2,Sol_Breakin_3], output_variables3,label,variable_limits='tight') #     
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

In [10]:
label = ["2","3"] 
output_variables3 = [
                "Positive particle surface concentration",
                "Negative particle surface concentration",
                "X-averaged electrolyte potential",
                "X-averaged electrolyte potential [V]"
            ]
quick_plot = pb.QuickPlot([Sol_Breakin_2,Sol_Breakin_3], output_variables3,label,variable_limits='tight') #     
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

In [11]:
label = ["2", "3"] 
output_variables3 = [
                "Negative electrode roughness ratio",
                "Positive electrode roughness ratio",
                "Total SEI on cracks thickness [m]",
                "X-averaged total SEI on cracks thickness [m]", 
                "X-averaged negative electrode roughness ratio",
                "Negative electrode capacity [A.h]",
                "Positive electrode capacity [A.h]",
            ]
quick_plot = pb.QuickPlot([Sol_Breakin_2,Sol_Breakin_3], output_variables3,label,variable_limits='tight') #     
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…